In [1]:
import pandas as pd
import sqlite3

In [2]:
def get_most_common_positions(conn: str) -> pd.DataFrame:
    """
    Retrieve the most common starting and ending positions for each driver in each year.
    
    :param conn: The database connection.
    :return: DataFrame containing driverId, year, most_common_starting_position, and most_common_ending_position.
    """
    sql_query = '''
    WITH StartingPositionCounts AS (
        SELECT
            driverId,
            raceId,
            grid,
            COUNT(*) AS starting_position_count
        FROM
            results
        GROUP BY
            driverId,
            raceId,
            grid
    ),
    EndingPositionCounts AS (
        SELECT
            driverId,
            raceId,
            position,
            COUNT(*) AS ending_position_count
        FROM
            results
        GROUP BY
            driverId,
            raceId,
            position
    )
    SELECT
        r.driverId,
        ra.year,
        r.grid AS most_common_starting_position,
        r.position AS most_common_ending_position
    FROM
        results r
    JOIN
        races ra ON r.raceId = ra.raceId
    JOIN StartingPositionCounts start_counts ON r.driverId = start_counts.driverId
        AND r.raceId = start_counts.raceId
        AND r.grid = start_counts.grid
    JOIN EndingPositionCounts end_counts ON r.driverId = end_counts.driverId
        AND r.raceId = end_counts.raceId
        AND r.position = end_counts.position
    WHERE
        ra.year <> 2023
        AND start_counts.starting_position_count = (
            SELECT
                MAX(starting_position_count)
            FROM
                StartingPositionCounts
            WHERE
                driverId = r.driverId
                AND raceId = r.raceId
        )
        AND end_counts.ending_position_count = (
            SELECT
                MAX(ending_position_count)
            FROM
                EndingPositionCounts
            WHERE
                driverId = r.driverId
                AND raceId = r.raceId
        )
    GROUP BY
        r.driverId, ra.year;
    '''
    df = pd.read_sql_query(sql_query, conn)

    return df
